Install relevant modules and packages 

In [ ]:
!apt-get install -y dcm2niix
!apt-get install -y freesurfer # note that this may take a while
!apt-get install -y parallel

!pip install nibabel
!pip install glob

Mount the google drive so that new files can be created (they will be deleted in the last cell)

from google.colab import drive
drive.mount('/content/drive')

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files

In [ ]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/drive/MyDrive/nii_files
!mkdir /content/drive/MyDrive/freesurfer_output

Convert the DICOMs to NIfTIs

In [ ]:
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/drive/MyDrive/nii_files'

!dcm2niix '{INPUT_PATH}' --OutDirMode=1 --OutDir='{OUTPUT_PATH}' > /content/drive/MyDrive/log_file.txt

Reconstruct the Nii files.

Parallel is used here for demonstration purposes, but since we only have a single dicom-containing folder, it doesn't do anything in practice. 

Using parallel in this way allows you to recon multiple subjects simultaneously by recruiting n CPU cores (determined by --jobs n)

In [ ]:
RECON_PATH = '/content/drive/MyDrive/freesurfer_output'

import glob
ALL_NIIS = glob.glob('/content/drive/MyDrive/nii_files/*')

parallel --jobs 1 'recon-all -i {} -sd '"$RECON_PATH"' -all' ::: $ALL_NIIS